In [1]:
import quandl
from datetime import date
from bokeh.charts import Bar
from bokeh.io import output_notebook, show, vform
output_notebook()
from pandas import ExcelWriter

Loading BokehJS ...

__plan and notes__

- get physical volume from USDA convert to contracts

- calc total volume traded last year divided by phy volume

- calc average OI for the year divide by phys volume

- if possible, calc total deliveries for the year divide by phys volume

do 1. 2. 3. for producer/merchant/processor

Notes:

- [oats specs](www.cmegroup.com/trading/agricultural/grain-and-oilseed/oats_contract_specifications.html)
- cot dataset titles like "Commitment of Traders - Oats - Futures Only"
 - dept of ag NASS dataset titles like "Oats - Production, Measured In Bu"

- [CME contract specs](http://www.cmegroup.com/trading/agricultural/grain-and-oilseed/oats_contract_specifications.html) example

- Some of the 2015 crop data is missing, I'm filling in 2015 with infromation form this pdf, i dont know why 2015 isnt online yet, but other years are the same in this report as the dataset. ["Crop Production 2015 Summary"](http://www.usda.gov/nass/PUBS/TODAYRPT/cropan16.pdf) [didnt end up doing this but could be done in future revision]
    
- 3,929,885 - 2015 Soybeans


__Set up and Pull data__

In [2]:
#SET UP FOR QUANDL PULL

#volume is from wiki continous futures
# .7 is the volume column
Volume_codes = ["CHRIS/CME_O1.7", #oats 1
            "CHRIS/CME_O2.7", #oats 2
            "CHRIS/CME_O3.7", #oats 3 
                
            "CHRIS/CME_S1.7", #soybean 1
            "CHRIS/CME_S2.7", #soybean 2
            "CHRIS/CME_S3.7", #soybean 3
            "CHRIS/CME_S4.7", #soybean 4
            "CHRIS/CME_S5.7", #soybean 5
            "CHRIS/CME_S6.7", #soybean 6
            "CHRIS/CME_S7.7", #soybean 7
            "CHRIS/CME_S8.7", #soybean 8
            "CHRIS/CME_S9.7", #soybean 9
            "CHRIS/CME_S10.7", #soybean 10
            
            "CHRIS/CME_C1.7", #corn 1
            "CHRIS/CME_C2.7", #corn 2
            "CHRIS/CME_C3.7", #corn 3
            "CHRIS/CME_C4.7", #corn 4
            "CHRIS/CME_C5.7", #corn 5
            "CHRIS/CME_C6.7", #corn 6
            "CHRIS/CME_C7.7", #corn 7
            "CHRIS/CME_C8.7", #corn 8
            "CHRIS/CME_C9.7", #corn 9
            "CHRIS/CME_C10.7", #corn 10
                
            "CHRIS/CME_W1.7", #Wheat (all KC) 1
            "CHRIS/CME_W2.7", #Wheat 2
            "CHRIS/CME_W3.7", #Wheat 3
            "CHRIS/CME_W4.7", #Wheat 4
            "CHRIS/CME_W5.7", #Wheat 5
            "CHRIS/CME_W6.7", #Wheat 6
            "CHRIS/CME_W7.7", #Wheat 7
            "CHRIS/CME_W8.7", #Wheat 8

            "CHRIS/CME_RR1.7", #rice 1
            "CHRIS/CME_RR2.7", #rice 2
            "CHRIS/CME_RR3.7", #rice 3       
           ]

#oi from COT
OI_codes = ["CFTC/O_F_ALL", #oats
            "CFTC/S_F_ALL", #soybean
            "CFTC/C_F_ALL", #corn
            "CFTC/W_F_ALL", #wheat
            "CFTC/RR_F_ALL", #rice
           ]

#production from USDA NASS (National Ag Stats Service)
Prod_codes = ["USDANASS/NASS_OATSPRODUCTIONMEASUREDINBU", #oats
              "USDANASS/NASS_SOYBEANSPRODUCTIONMEASUREDINBU", #soybean
              "USDANASS/NASS_CORNGRAINPRODUCTIONMEASUREDINBU", #corn
              "USDANASS/NASS_WHEATWINTERREDSOFTPRODUCTIONMEASUREDINBU", #wheat
              "USDANASS/NASS_RICEPRODUCTIONMEASUREDINCWT", #rice
             ]

#four years ago for quandl pull
fouryears = date(2012, 1, 1)

In [4]:
OI = quandl.get(OI_codes, trim_start=fouryears, authtoken="UDaCz3fqxxJUAL3-WWmD", returns="pandas")

In [5]:
Production = quandl.get(Prod_codes, trim_start=fouryears, authtoken="UDaCz3fqxxJUAL3-WWmD", returns="pandas")

In [6]:
Volume = quandl.get(Volume_codes,trim_start=fouryears, authtoken="UDaCz3fqxxJUAL3-WWmD", returns="pandas")

__Production Data__

In [10]:
Production.head()

,USDANASS/NASS_OATSPRODUCTIONMEASUREDINBU - Value,USDANASS/NASS_SOYBEANSPRODUCTIONMEASUREDINBU - Value,USDANASS/NASS_CORNGRAINPRODUCTIONMEASUREDINBU - Value,USDANASS/NASS_WHEATWINTERREDSOFTPRODUCTIONMEASUREDINBU - Value,USDANASS/NASS_RICEPRODUCTIONMEASUREDINCWT - Value
Date,,,,,
2012-12-31,61486000.0,3.042044e+09,1.075511e+10,412729000.0,199939000.0
2013-12-31,64642000.0,3.357984e+09,1.382896e+10,568481000.0,189953000.0
2014-12-31,69684000.0,3.968823e+09,1.421553e+10,455297000.0,221035000.0
2015-12-31,83640000.0,NaN,NaN,393416000.0,NaN


In [7]:
#PRODUCTION
#copy because quandl has a call limit and sometimes and I want this dataset to be my base.
prod = Production.copy()

#index based on date
prod.index.name = None

#already had this code thought it'd be easier than cut and paste them ugly col names
oat_cols = [col for col in prod.columns if 'OATS' in col]
soybean_cols = [col for col in prod.columns if 'SOYBEAN' in col]
corn_cols = [col for col in prod.columns if 'CORN' in col]
wheat_cols = [col for col in prod.columns if 'WHEAT' in col]
rice_cols = [col for col in prod.columns if 'RICE' in col]

#convert phy volum produced into contracts
#contract sizes from CME website
prod['Contracts Oats'] = prod[oat_cols]/5000
prod['Contracts Soybean'] = prod[soybean_cols]/5000
prod['Contracts Corn'] = prod[corn_cols]/5000
prod['Contracts Wheat'] = prod[wheat_cols]/5000
prod['Contracts Rice'] = prod[rice_cols]/2000

In [13]:
prod = prod.drop(oat_cols,1)
prod = prod.drop(soybean_cols,1)
prod = prod.drop(corn_cols,1)
prod = prod.drop(wheat_cols,1)
prod = prod.drop(rice_cols,1)

__OI Data__

In [12]:
#OI
oi = OI.copy()

#index based on date
oi.index.name = None

#resample to yearly data, take mean of colums
oi = oi.resample('A').mean()

In [14]:
#keep certain columns
oi = oi[['CFTC/O_F_ALL - Open Interest',
       'CFTC/O_F_ALL - Producer/Merchant/Processor/User Longs',
       'CFTC/O_F_ALL - Producer/Merchant/Processor/User Shorts',
         
        'CFTC/S_F_ALL - Open Interest',
       'CFTC/S_F_ALL - Producer/Merchant/Processor/User Longs',
       'CFTC/S_F_ALL - Producer/Merchant/Processor/User Shorts',
         
        'CFTC/C_F_ALL - Open Interest',
       'CFTC/C_F_ALL - Producer/Merchant/Processor/User Longs',
       'CFTC/C_F_ALL - Producer/Merchant/Processor/User Shorts',
         
        'CFTC/W_F_ALL - Open Interest',
       'CFTC/W_F_ALL - Producer/Merchant/Processor/User Longs',
       'CFTC/W_F_ALL - Producer/Merchant/Processor/User Shorts',
         
        'CFTC/RR_F_ALL - Open Interest',
       'CFTC/RR_F_ALL - Producer/Merchant/Processor/User Longs',
       'CFTC/RR_F_ALL - Producer/Merchant/Processor/User Shorts',
        ]]

In [15]:
#rename columns
oi.columns = ['Oats Open Interest',
             'Oats Producer Longs',
             'Oats Producer Shorts',
              
             'Soybean Open Interest',
             'Soybean Producer Longs',
             'Soybean Producer Shorts',
              
             'Corn Open Interest',
             'Corn Producer Longs',
             'Corn Producer Shorts',
              
             'Wheat Open Interest',
             'Wheat Producer Longs',
             'Wheat Producer Shorts',
              
             'Rice Open Interest',
             'Rice Producer Longs',
             'Rice Producer Shorts'
             ]

__Volume Data__

In [16]:
#VOLUME
vol = Volume.copy()

In [17]:
#index based on date
vol.index.name = None

In [18]:
oat_c = [col for col in vol.columns if 'CME_O' in col]
soybean_c = [col for col in vol.columns if 'CME_S' in col]
corn_c = [col for col in vol.columns if 'CME_C' in col]
wheat_c = [col for col in vol.columns if 'CME_W' in col]
rice_c = [col for col in vol.columns if 'CME_R' in col]

In [19]:
vol['oat TV'] = vol[oat_c].sum(axis=1)
vol['soybean TV'] = vol[soybean_c].sum(axis=1)
vol['corn TV'] = vol[corn_c].sum(axis=1)
vol['wheat TV'] = vol[wheat_c].sum(axis=1)
vol['rice TV'] = vol[rice_c].sum(axis=1)

In [20]:
vol = vol.resample('A').sum()

In [21]:
vol = vol[['oat TV','soybean TV','corn TV','wheat TV','rice TV']]

__ Join datasets make columns __

In [22]:
vol = vol[vol.index.year != 2016]
oi = oi[oi.index.year != 2016]

In [23]:
pv = prod.join(vol)

In [24]:
pvo = pv.join(oi)

In [25]:
pvo

,Contracts Oats,Contracts Soybean,Contracts Corn,Contracts Wheat,Contracts Rice,oat TV,soybean TV,corn TV,wheat TV,rice TV,...,Soybean Producer Shorts,Corn Open Interest,Corn Producer Longs,Corn Producer Shorts,Wheat Open Interest,Wheat Producer Longs,Wheat Producer Shorts,Rice Open Interest,Rice Producer Longs,Rice Producer Shorts
2012-12-31,12297.2,608408.8,2151022.2,82545.8,99969.5,285186.0,53494064.0,72680440.0,28478378.0,390937.0,...,384333.584906,1.222394e+06,235429.849057,624896.754717,448913.811321,40155.264151,162945.735849,14592.754717,5542.622642,6577.245283
2013-12-31,12928.4,671596.8,2765792.8,113696.2,94976.5,253422.0,46838774.0,63948994.0,24891308.0,280705.0,...,311123.884615,1.211489e+06,271071.711538,484775.788462,413347.442308,64938.519231,117978.634615,11268.442308,5235.076923,5801.884615
2014-12-31,13936.8,793764.6,2843106.4,91059.4,110517.5,189443.0,48878387.0,68642593.0,25766260.0,216178.0,...,283839.538462,1.312403e+06,280910.153846,580553.846154,395130.384615,48378.538462,119332.615385,8856.384615,4060.942308,3842.903846
2015-12-31,16728.0,NaN,NaN,78683.2,NaN,173867.0,52232952.0,80924197.0,30599534.0,303870.0,...,302407.884615,1.333002e+06,305302.403846,562958.576923,402690.865385,50613.038462,110065.653846,10879.846154,5236.442308,3211.230769


In [26]:
pvo['year'] = pvo.index.years

In [27]:
#oats plot
op = Bar(pvo, 'year', values='Contracts Oats',
        title="Total DISPL by YR", bar_width=0.4)

In [28]:
show(op)

In [ ]:
writer = ExcelWriter('output.xlsx')
pvo.to_excel(writer,'Sheet1')
writer.save()